### Runs an extended Kalman filter using IHME SEIIR predictions along with measurement data of confirmed Covid-19 case counts (from New York Times data) and CMU/Facebook symptom data (from Covid-19 Symptom Challenge) to generate updated 7-day predictions of case counts for counties in New York State and Florida.

Developed by the University of Washington team of Les Atlas, Abraham Flaxman and Michael Rhoads.

S - Susceptible
E - Exposed
I1 - Presymptomatic
I2 - Symptomatic
R - Recovered

In [81]:
#%load_ext autoreload
#%autoreload

In [82]:
#%reset

In [83]:
import math
import datetime

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import data_sets
import seiir_compartmental

In [84]:
# functions to support the Kalman filtering
def get_predicts_prior(day, seiir):
    x_hat = np.array([[seiir['S'].loc[day]],
                      [seiir['E'].loc[day]],
                      [seiir['I1'].loc[day]],
                      [seiir['I2'].loc[day]],
                      [seiir['R'].loc[day]]])

    beta_k = seiir['beta'].loc[day]

    return x_hat, beta_k


def step_seiir(x_hat, constants, beta_k, days=7):
    s_dict = {'S': x_hat[0, 0],
              'E': x_hat[1, 0],
              'I1': x_hat[2, 0],
              'I2': x_hat[3, 0],
              'R': x_hat[4, 0]}

    s = pd.Series(s_dict)

    for i in range(days):
        infectious = s.loc['I1'] + s.loc['I2']
        s = seiir_compartmental.compartmental_covid_step(s, s.sum(),
                                                         infectious,
                                                         constants['alpha'],
                                                         beta_k,
                                                         constants['gamma1'],
                                                         constants['gamma2'],
                                                         constants['sigma'],
                                                         constants['theta'])
    x_hat_future_prior = np.array([[s.loc['S']],
                                   [s.loc['E']],
                                   [s.loc['I1']],
                                   [s.loc['I2']],
                                   [s.loc['R']]])

    return x_hat_future_prior


def predict_step(x_hat_k1_prior, P, Q, beta_k, constants):
    S = x_hat_k1_prior[0, 0]
    E = x_hat_k1_prior[1, 0]
    I1 = x_hat_k1_prior[2, 0]
    I2 = x_hat_k1_prior[3, 0]
    R = x_hat_k1_prior[4, 0]
    N = S + E + I1 + I2 + R
    alpha = constants['alpha']
    sigma = constants['sigma']
    gamma1 = constants['gamma1']
    gamma2 = constants['gamma2']

    part_f_S = np.array([[-beta_k * math.pow(I1 + I2, alpha) / N],
                         [beta_k * math.pow(I1 + I2, alpha) / N],
                         [0],
                         [0],
                         [0]])

    part_f_E = np.array([[0],
                         [-sigma],
                         [sigma],
                         [0],
                         [0]])

    part_f_I1 = np.array([[-alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [-gamma1],
                          [gamma1],
                          [0]])

    part_f_I2 = np.array([[-alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [0],
                          [-gamma2],
                          [gamma2]])
    
    part_f_R = np.array([[0],
                         [0],
                         [0],
                         [0],
                         [0]])

    # 5x5
    f_jacob = np.concatenate([part_f_S, part_f_E, part_f_I1, part_f_I2,
                              part_f_R], axis=1)


    # 5x5
    # P_prior = f_jacob * P * f_jacob^T + Q
    P_prior = np.matmul(np.matmul(f_jacob, P), f_jacob.T) + Q
    
    return P_prior


def update_step(x_hat, x_hat_k1, P_prior, Rn, rho1, rho2, z_k, measure):
    # 5x5
    ep = 10**-8
    
    # if there is no survey measurement used, then rho1 is not needed
    if measure is None:
        H = np.array([[ep, 0, 0, 0, 0],
                      [0, ep, 0, 0, 0],
                      [0, 0, ep, 0, 0],
                      [0, 0, 0, rho2, 0],
                      [0, 0, 0, 0, ep]])
    else:
                
        H = np.array([[ep, 0, 0, 0, 0],
                      [0, ep, 0, 0, 0],
                      [0, 0, ep, rho1, 0],
                      [0, 0, 0, rho2, 0],
                      [0, 0, 0, 0, ep]])
    
    
    # Si = H * P_prior * H^T + Rn
    Si = np.matmul(np.matmul(H, P_prior), H.T) + Rn

    # K = P_prior * H.T (H * P_prior * H.T + R)^-1
    # K_new = P_prior * H^T * Si^(-1)
    K_new = np.matmul(np.matmul(P_prior, H.T), np.linalg.inv(Si))
    
    y_new = np.matmul(H, x_hat)

    # 5x1
    diff = z_k - y_new

    x_hat_k1_post = x_hat_k1 + np.matmul(K_new, diff)

    P_post = P_prior - np.matmul(np.matmul(K_new, Si), K_new.T)


    return x_hat_k1_post, P_post, diff, K_new[3, 3]


def create_data_sets():
    seiir_fl = pd.read_csv(r'data/seiir_projections/florida_proj.csv', header=0,
                            index_col='date', parse_dates=True)

    seiir_ny = pd.read_csv(r'data/seiir_projections/new_york_proj.csv', header=0,
                            index_col='date', parse_dates=True)
    
    smell_data = data_sets.create_symptom_df()
    
    case_data = data_sets.create_case_df_county()
    
    return seiir_fl, seiir_ny, smell_data, case_data


def get_smell_data(fips, fb_data):

    if fips == 'New York City':
        nyc_fips = ['36005', '36061', '36047', '36085']
        fb_data_geo = fb_data.loc[(slice(None), '36081'), :].copy()
        fb_data_geo = fb_data_geo.mean(level='date')

        for borough in nyc_fips:
            fb_data_geo += fb_data.loc[(slice(None), borough), :].copy().mean(level='date')

    else:
        fb_data_geo = fb_data.loc[(slice(None), fips), :].copy()

        # collapse down to a single index column (date)
        fb_data_geo.index = fb_data_geo.index.droplevel([0, 1])

    return fb_data_geo
    


def calc_fb_ma7(fb_data):
    """
    Returns a Pandas series
    """
    # the fb_data is a DataFrame while the case_data is a Series
    fb_ma7 = fb_data.rolling(window=7).mean()
    fb_ma7 = fb_ma7.iloc[6:, :]
    prop_ma7 = fb_ma7['num_stl'].div(fb_ma7['n'])

    return prop_ma7, fb_ma7['n'].copy(), fb_ma7['num_stl'].copy()


def calc_mse(prediction, actual):
    """
    Inputs should be two Pandas Series of same length.
    Outputs a float.
    """
    err = prediction - actual
    sum_sq_err = (err**2).sum()
    mse = sum_sq_err / err.count()
    return mse


def create_hh_data():
    # bring in household covid symptom data
    full_data = pd.read_csv(r'data/fb_hh_cli.csv', header=0, dtype={'fips': 'str',
                                                                    'n': 'int64',
                                                                    'hh_cli': 'float64'},
                                                            parse_dates=[2])
    full_data['fips'] = full_data['fips'].str.replace('(^[0-9]{4}\.0)', '0\g<1>')
    full_data['fips'] = full_data['fips'].str.extract('(^[0-9]{5})')
    time_start = full_data['date'].min()
    time_end = full_data['date'].max()
    full_data.set_index(['fips', 'date'], inplace=True)
    full_data.sort_index(inplace=True)

    # derive count of survey respondents with household members having covid symptoms
    full_data['num_hh_cli'] = full_data['n'].mul(full_data['hh_cli']).round()
    full_data['num_hh_cli'] = full_data['num_hh_cli'].astype('int64')

    # group by county and date
    data_of_interest = full_data[['n', 'num_hh_cli']].copy().groupby(level=(0, 1)).sum()
    idx = pd.IndexSlice

    # create full date range
    date_rng = pd.date_range(time_start, time_end)
    iterables = [data_of_interest.index.levels[0], date_rng]
    new_index = pd.MultiIndex.from_product(iterables, names=['fips', 'date'])

    data_of_interest = data_of_interest.reindex(index=new_index)
    # this will have NaN values in the new index entries for which there was no
    # previous data -- fill them upon extracting a particular county
    
    return data_of_interest


def create_county_lists():
    case_data = data_sets.create_case_df_county()
    ny_counties = case_data[case_data['state'] == 'New York']['county'].unique()
    fl_counties = case_data[case_data['state'] == 'Florida']['county'].unique()
    return ny_counties, fl_counties

In [85]:
# create data sets

seiir_fl, seiir_ny, smell_data, case_data = create_data_sets()
hh_data = create_hh_data()

In [86]:
seiir_ny.index[-1]


Timestamp('2021-12-31 00:00:00')

In [87]:

def run_county(the_state, county_name, seiir, smell_data, hh_data, case_data, measure='smell', K0=datetime.date(2020, 4, 18), delay=6):
    """
    The main function that calculates the forecast for a given county over the entire time range
    Arguments:
        measure -- potential values are: 'smell', 'hh', or None
    """
    # set constants
    #K0 = datetime.date(2020, 4, 12)

    if county_name == 'New York City':
        the_county = county_name
    else:
        the_county = data_sets.get_fips(the_state, county_name)

    constants = {
        'alpha': 0.948786,
        'gamma1': 0.500000,
        'gamma2': 0.662215,
        'sigma': 0.266635,
        'theta': 6.000000
        }

    # set initial values for Kalman filter parameters
    P_mult = 1
    Q_mult = 1

    # Rn is the R noise covariance matrix; it remains constant thru the stepping of the
    # Kalman filter
    Rn_mult = 5*10**-8

    Rn_22 = 10000
    Rn_32 = 1000

    Rn_23 = 1000
    Rn_33 = 100

    Rn = Rn_mult * np.array([[0, 0, 0, 0, 0],
                             [0, 0, 0, 0, 0],
                             [0, 0, Rn_22, Rn_23, 0],
                             [0, 0, Rn_32, Rn_33, 0],
                             [0, 0, 0, 0, 0]])

    Q = Q_mult * np.eye(5)
    P = P_mult * np.eye(5)
    
    if the_county == 'New York City':
        county_pop = 0
        for each in ['36081', '36005', '36061', '36047', '36085']:
            this_count, state_pop = data_sets.get_pops(each)
            county_pop += this_count
    else:
        county_pop, state_pop = data_sets.get_pops(the_county)

    b = county_pop / state_pop

    # generate data
    case_data_geo = case_data.loc[the_county]['case_rate'].copy()
    smell_data_geo = get_smell_data(the_county, smell_data)
    

    if measure == 'hh':
        idx = pd.IndexSlice
        
        if the_county == 'New York City':
            nyc_fips = ['36005', '36061', '36047', '36085']
            hh_cli = hh_data.loc[idx['36081', :], :].loc['36081'].copy()
            hh_cli.fillna(method='pad', inplace=True)

            for borough in nyc_fips:
                bor_hh_cli = hh_data.loc[idx[borough, :], :].loc[borough].copy()
                bor_hh_cli.fillna(method='pad', inplace=True)
                hh_cli += bor_hh_cli

        else:
            hh_cli = hh_data.loc[idx[the_county, :], :].loc[the_county].copy()
            hh_cli.fillna(method='pad', inplace=True)

        # calculate moving averages on the fb and case data
        hh_cli_ma7 = hh_cli.rolling(window=7).mean()
        hh_cli_ma7 = hh_cli_ma7.iloc[6:, :]
        num_survey_ma7 = hh_cli_ma7['num_hh_cli']
        prop_cli_ma7 = num_survey_ma7.div(hh_cli_ma7['n'])
        
    elif measure == 'smell':
        prop_ma7, n_ma7, num_survey_ma7 = calc_fb_ma7(smell_data_geo)
        
    else:
        num_survey_ma7 = None


    case_ma7 = case_data_geo.rolling(window=7).mean()
    case_ma7_all = case_ma7.iloc[6:]
    

    # get starting compartment values for the state level
    x_hat_state_k0, beta_k0 = get_predicts_prior(K0, seiir)

    # convert to the county level
    x_hat_k0 = b * x_hat_state_k0

    I2_county = x_hat_k0[3, 0]


    rho1 = .05
    rho2 = case_ma7_all.loc['2020-04-12'] / I2_county


    # create empty dictionaries to hold the estimated values
    case_est = {}
    seiir_pred = {}

    diff_rat = {}

    K_val_dict = {}


    # Original data run ----------------
    start = K0
    k = start

    while k <= datetime.date(2020, 12, 8):
        # previous constraint was 2020-10-23

    # each cycle of the while loop executes a step

        # get state level compartments
        x_hat_state_k, beta_k = get_predicts_prior(k, seiir)

        # step the state level compartments 1 day forward
        x_hat_state_k1 = step_seiir(x_hat_state_k, constants, beta_k, days=1)
        P = predict_step(x_hat_state_k, P, Q, beta_k, constants)
        
        # then 6 more
        P_est = P.copy()
        for i in range(6):
            x_hat_est = step_seiir(x_hat_state_k1, constants, beta_k, days=1)
            P_est = predict_step(x_hat_state_k1, P_est, Q, beta_k, constants)
            x_hat_state_k1 = x_hat_est.copy()

        # convert the state level compartments to county level values
        x_hat_k = b * x_hat_state_k
        x_hat_k1 = b * x_hat_state_k1

        
        indexDate = k + datetime.timedelta(days=7)
        # store seiir prediction before it's modified by Kalman filter
        # multiply by rho2 to get values equivalent to confirmed case count
        seiir_pred[indexDate] = rho2 * x_hat_k1[3, 0]

        # get measurements for current day
        if measure == 'smell':
            z_k = np.array([[0],
                            [0],
                            [prop_ma7.loc[k - datetime.timedelta(days=delay)]],
                            [case_ma7_all.loc[k]],
                            [0]])    

        elif measure == 'hh':
            z_k = np.array([[0],
                            [0],
                            [prop_cli_ma7.loc[k - datetime.timedelta(days=delay)]],
                            [case_ma7_all.loc[k]],
                            [0]])
            
        else:
            z_k = np.array([[0],
                            [0],
                            [0],
                            [case_ma7_all.loc[k]],
                            [0]])

        # predict step using the stepped fwd SEIIR compartment values 
        #P = predict_step(x_hat_k1, P, Q, beta_k, constants)

        # update step
        x_hat_post, P_post, the_diff, K_val = update_step(x_hat_k, x_hat_k1, P_est, Rn,
                                         rho1, rho2, z_k, measure)


        # store estimated values for proportion and case rate
        K_val_dict[indexDate] = K_val
        case_est[indexDate] = rho2 * x_hat_post[3, 0]


        diff_rat[indexDate] = the_diff[2, 0] / the_diff[3, 0]

        # update the P and k
        # comment out the P update because it has already been stepped once
        #P = P_post
        k += datetime.timedelta(days=1)

    # create pandas series of the estimated case rate
    predicted_case = pd.Series(case_est)
    predicted_seiir_prior = pd.Series(seiir_pred)
    K_val_series = pd.Series(K_val_dict)
    
    return predicted_case, predicted_seiir_prior, case_ma7_all, num_survey_ma7, K_val_series

In [89]:
# a single county run
predicted_case, predicted_seiir_prior, case_ma7_all, num_survey_ma7, K_vals = run_county('NY', 'Nassau', seiir_ny, smell_data, hh_data, case_data, measure='hh', K0=datetime.date(2020, 4, 18), delay=0)

In [160]:
K0 = datetime.date(2020, 4, 18)

ny_counties, fl_counties = create_county_lists()

In [162]:
fl_counties

array(['Alachua', 'Baker', 'Bay', 'Bradford', 'Brevard', 'Broward',
       'Calhoun', 'Charlotte', 'Citrus', 'Clay', 'Collier', 'Columbia',
       'DeSoto', 'Dixie', 'Duval', 'Escambia', 'Flagler', 'Franklin',
       'Gadsden', 'Gilchrist', 'Glades', 'Gulf', 'Hamilton', 'Hardee',
       'Hendry', 'Hernando', 'Highlands', 'Hillsborough', 'Holmes',
       'Indian River', 'Jackson', 'Jefferson', 'Lafayette', 'Lake', 'Lee',
       'Leon', 'Levy', 'Liberty', 'Madison', 'Manatee', 'Marion',
       'Martin', 'Miami-Dade', 'Monroe', 'Nassau', 'Okaloosa',
       'Okeechobee', 'Orange', 'Osceola', 'Palm Beach', 'Pasco',
       'Pinellas', 'Polk', 'Putnam', 'St. Johns', 'St. Lucie',
       'Santa Rosa', 'Sarasota', 'Seminole', 'Sumter', 'Suwannee',
       'Taylor', 'Union', 'Volusia', 'Wakulla', 'Walton', 'Washington'],
      dtype=object)

In [101]:
prediction_index = predicted_case.index


## Forecast NY State counties

In [112]:
shifted_range = pd.date_range(start=left, end=right)

In [105]:
len(shifted_range)

228

In [163]:
# conduct a forecast for all the counties in NY that there exists survey data for
state_list = ['NY', 'FL']

left = K0 + datetime.timedelta(days=7)
right = datetime.date(2020, 12, 8) + datetime.timedelta(days=7)
shifted_range = pd.date_range(start=left, end=right)
numLoop = 5

for state in state_list:
    df_index = []
    err_data = []

    predictions_case_only = pd.DataFrame()
    predictions_smell_0delay = pd.DataFrame()
    predictions_hh_0delay = pd.DataFrame()

    predictions_ihme = pd.DataFrame()
    predictions_naive = pd.DataFrame()
    
    
    if state == 'NY':
        seiir_data = seiir_ny
        county_list = ny_counties
    else:
        seiir_data = seiir_fl
        county_list = fl_counties

    for each in county_list:
        if each == 'New York City':
            fips = 'New York City'
        else:
            fips = data_sets.get_fips(state, each)
            if (fips not in hh_data.index.levels[0]) or (fips not in smell_data.index.levels[1]):
                continue

        print('Starting', fips)

        # dictionary to hold error for each forecast: our 3 versions, ihme, and naive
        county_data = {}


        # pandas series to hold the predictions that will be averaged
        conf_avg = pd.Series(data=np.zeros(len(prediction_index)), index=prediction_index, dtype='float64')
        ihme_avg = pd.Series(data=np.zeros(len(prediction_index)), index=prediction_index, dtype='float64')
        for i in range(numLoop):
            predicted_case, predicted_seiir_prior, case_ma7_all, num_survey_ma7, K_vals = run_county(state, each, seiir_data, smell_data, hh_data, case_data, measure=None, K0=K0, delay=0)
            conf_avg += predicted_case
            ihme_avg += predicted_seiir_prior
        # store the averaged predictions in their DataFrames (ihme only needs to be done once)
        predictions_case_only[fips] = conf_avg / numLoop
        predictions_ihme[fips] = ihme_avg / numLoop
        # store the mean mse in the dictionary
        county_data['mse_confirmed-only'] = calc_mse(predictions_case_only[fips].loc[left:right], case_ma7_all.loc[left:right])
        county_data['ihme'] = calc_mse(predictions_ihme[fips].loc[left:right], case_ma7_all.loc[left:right])



        smell0_avg = pd.Series(data=np.zeros(len(prediction_index)), index=prediction_index, dtype='float64')
        for i in range(numLoop):
            predicted_case, predicted_seiir_prior, case_ma7_all, num_survey_ma7, K_vals = run_county(state, each, seiir_data, smell_data, hh_data, case_data, measure='smell', K0=K0, delay=0)
            smell0_avg += predicted_case
        predictions_smell_0delay[fips] = smell0_avg / numLoop
        county_data['mse_smell-0delay'] = calc_mse(predictions_smell_0delay[fips].loc[left:right], case_ma7_all.loc[left:right])


        hh0_avg = pd.Series(data=np.zeros(len(prediction_index)), index=prediction_index, dtype='float64')
        for i in range(numLoop):
            predicted_case, predicted_seiir_prior, case_ma7_all, num_survey_ma7, K_vals = run_county(state, each, seiir_data, smell_data, hh_data, case_data, measure='hh', K0=K0, delay=0)
            hh0_avg += predicted_case
        predictions_hh_0delay[fips] = hh0_avg / numLoop
        county_data['mse_hh-0delay'] = calc_mse(predictions_hh_0delay[fips].loc[left:right], case_ma7_all.loc[left:right])


        # naive prediction
        naive_pred = case_ma7_all.loc[left-datetime.timedelta(days=7):right-datetime.timedelta(days=7)].copy()
        naive_pred.index = shifted_range
        # store for future use
        predictions_naive[fips] = naive_pred
        # compute error
        county_data['naive'] = calc_mse(naive_pred.loc[left:right], case_ma7_all.loc[left:right])


        df_index.append(fips)
        err_data.append(county_data)
        print('Complete with', fips)
        
    err_df = pd.DataFrame(err_data, index=df_index)
    if state == 'NY':
        err_df.to_csv(r'output/err_ny_20201208.csv')
        predictions_case_only.to_csv(r'output/pred_ny_case_only.csv')
        predictions_smell_0delay.to_csv(r'output/pred_ny_smell_0delay.csv')
        predictions_hh_0delay.to_csv(r'output/pred_ny_hh_0delay.csv')

        predictions_ihme.to_csv(r'output/pred_ny_ihme.csv')
        predictions_naive.to_csv(r'output/pred_ny_naive.csv')
    else:
        err_df.to_csv(r'output/err_fl_20201208.csv')
        predictions_case_only.to_csv(r'output/pred_fl_case_only.csv')
        predictions_smell_0delay.to_csv(r'output/pred_fl_smell_0delay.csv')
        predictions_hh_0delay.to_csv(r'output/pred_fl_hh_0delay.csv')

        predictions_ihme.to_csv(r'output/pred_fl_ihme.csv')
        predictions_naive.to_csv(r'output/pred_fl_naive.csv')

Starting 36001
Complete with 36001
Starting 36003
Complete with 36003
Starting 36007
Complete with 36007
Starting 36009
Complete with 36009
Starting 36011
Complete with 36011
Starting 36013
Complete with 36013
Starting 36015
Complete with 36015
Starting 36017
Complete with 36017
Starting 36019
Complete with 36019
Starting 36021
Complete with 36021
Starting 36023
Complete with 36023
Starting 36025
Complete with 36025
Starting 36027
Complete with 36027
Starting 36029
Complete with 36029
Starting 36031
Complete with 36031
Starting 36033


C:\Users\mrhoa\miniconda3\envs\covid_DOA\lib\site-packages\ipykernel_launcher.py:185: RuntimeWarning: divide by zero encountered in double_scalars


Complete with 36033
Starting 36035
Complete with 36035
Starting 36037
Complete with 36037
Starting 36039
Complete with 36039
Starting 36041
Complete with 36041
Starting 36043
Complete with 36043
Starting 36045
Complete with 36045
Starting 36049
Complete with 36049
Starting 36051
Complete with 36051
Starting 36053
Complete with 36053
Starting 36055
Complete with 36055
Starting 36057
Complete with 36057
Starting 36059
Complete with 36059
Starting 36063
Complete with 36063
Starting 36065
Complete with 36065
Starting 36067
Complete with 36067
Starting 36069
Complete with 36069
Starting 36071
Complete with 36071
Starting 36073
Complete with 36073
Starting 36075
Complete with 36075
Starting 36077
Complete with 36077
Starting 36079
Complete with 36079
Starting 36083
Complete with 36083
Starting 36087
Complete with 36087
Starting 36089
Complete with 36089
Starting 36091
Complete with 36091
Starting 36093
Complete with 36093
Starting 36095
Complete with 36095
Starting 36097
Complete with 36097


In [164]:
len(predictions_hh_0delay[fips].loc[left:right])

235

# All plotting code below this

In [170]:

county = 'Westchester'
state_2l = 'NY'
if state_2l == 'NY':
    the_seiir = seiir_ny
else:
    the_seiir = seiir_fl


In [171]:
if county == 'New York City':
    county_pop = 0
    for each in ['36081', '36005', '36061', '36047', '36085']:
        this_count, state_pop = data_sets.get_pops(each)
        county_pop += this_count
else:
    county_pop, state_pop = data_sets.get_pops(data_sets.get_fips(state_2l, county))

In [172]:
county_pop

967506

In [173]:
state_pop

19453561

In [174]:


county_preds = {}
survey_data = {}
estimates = ['confirmed_only', 'hh_delay0', 'smell_delay0']
measure_type = [None, 'hh', 'smell']
#delay_list = [0, 0, 0]
for i in range(len(estimates)):
    predicted_case, predicted_seiir_prior, case_ma7_all, num_survey_ma7, K_vals = run_county(state_2l, county, the_seiir, smell_data, hh_data, case_data, measure=measure_type[i], K0=datetime.date(2020, 4, 18), delay=0)
    survey_data[estimates[i]] = num_survey_ma7
    county_preds[estimates[i]] = predicted_case


In [93]:
#estimate_type = ['hh_delay6', 'confirmed_only', 'hh_delay0', 'smell_delay0', 'smell_delay6']
labels = ['Forecast using Case Count Only', 'Forecast Case Count w/ Household Symptoms', 'Forecast Case Count w/ Loss of Smell/Taste']

In [ ]:
K0 = datetime.date(2020, 4, 18)
d = datetime.date(2020, 10, 23)
gold_right = d + datetime.timedelta(days=7)
tick_end = predicted_seiir_prior.index[-1]

week_interval = pd.date_range(start=K0, end=tick_end, freq='W')
week_interval = [x.to_pydatetime().date() for x in week_interval]

colors = ['blue', 'red', 'green']

for i in range(len(estimates)):
    plt.plot(county_preds[estimates[i]].index, county_preds[estimates[i]], label=labels[i], c=colors[i])
plt.plot(case_ma7_all.loc[K0:gold_right].index, case_ma7_all.loc[K0:gold_right], label='Actual Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior, label='IHME Forecast', c=)


#naieve estimate
naive_start = K0 + datetime.timedelta(days=7)
naive_d = d + datetime.timedelta(days=7)
plt.plot(case_ma7_all.loc[naive_start:naive_d].index, case_ma7_all.loc[K0:d], label='Naive Estimate', c='orange')
plt.legend(loc='upper center')
plt.xticks(rotation=30, ha='right', rotation_mode='anchor')
#plt.ylim(2700, 3055)
#plt.xlim(datetime.date(2020, 7, 25), datetime.date(2020, 7, 30))


#plt.title(county)
plt.ylabel('Number of Cases per Day')

In [ ]:
K_vals.head(50)

In [ ]:
plt.plot(K_vals.index, K_vals)
plt.title('Miami-Dade hh6 K[4, 4] Val')

In [43]:
xlim_left = datetime.date(2020, 6, 1)
xlim_right = None

leftylim_low = None
leftylim_high = None

rightylim_low = None
rightylim_high = None

xtick_size = 14
xlabel_size = 14

In [140]:
# plot findings -- multiple plots

# Plotting constants and variables ----------------

plt.style.use('seaborn-whitegrid')
matplotlib.rcParams.update({'font.size': 20})
purple = '#33016F'
bright_purp = '#9400D3'
light_purp = '#BF9BDE'
gold = '#9E7A27'
gray = '#797979'
bright_orange = '#FE5000'
width = 3
xtick_size = 14
xlabel_size = 14
%matplotlib qt


In [117]:

d = datetime.date(2020, 12, 8)
K0 = datetime.date(2020, 4, 18)
tick_end = predicted_seiir_prior.index[-1]
end = d + datetime.timedelta(days=7)

week_interval = pd.date_range(start=K0, end=tick_end, freq='W')
week_interval = [x.to_pydatetime().date() for x in week_interval]

#start = datetime.date(2020, 6, 1)

In [144]:
case_data = data_sets.create_case_df_county()

def get_case_count(case_df, state_2L, county):
    if county == 'New York City':
        fips = 'New York City'
    else:
        fips = data_sets.get_fips(state_2L, county)
    case_data_geo = case_df.loc[fips]['case_rate'].copy()
    case_ma7 = case_data_geo.rolling(window=7).mean()
    case_ma7_all = case_ma7.iloc[6:]
    return case_ma7_all

In [152]:
county = 'Miami-Dade'
state_2L = 'FL'
if county == 'New York City':
    fips = 'New York City'
else:
    fips = data_sets.get_fips(state_2L, county)
case_ma7_all = get_case_count(case_data, state_2L, county)

ihme_pred_all = pd.read_csv(r'output/pred_fl_ihme.csv', index_col=0, parse_dates=True)
case_only_pred_all = pd.read_csv(r'output/pred_fl_case_only.csv', index_col=0, parse_dates=True)
smell_pred_all = pd.read_csv(r'output/pred_fl_smell_0delay.csv', index_col=0, parse_dates=True)
hh_pred_all = pd.read_csv(r'output/pred_fl_hh_0delay.csv', index_col=0, parse_dates=True)
naive_pred_all = pd.read_csv(r'output/pred_fl_naive.csv', index_col=0, parse_dates=True)

ihme_pred = ihme_pred_all[fips]
case_only_pred = case_only_pred_all[fips]
smell_pred = smell_pred_all[fips]
hh_pred = hh_pred_all[fips]
naive_pred = naive_pred_all[fips]

In [153]:
ihme_pred


2020-04-25    401.939024
2020-04-26    397.863143
2020-04-27    398.946760
2020-04-28    396.130424
2020-04-29    388.652286
                 ...    
2020-12-11    987.801984
2020-12-12    980.318146
2020-12-13    983.041013
2020-12-14    984.962793
2020-12-15    988.115749
Name: 12086, Length: 235, dtype: float64

In [159]:
### This is the one plot

start = K0 + datetime.timedelta(days=7)
end = d + datetime.timedelta(days=7)

fig4, ax41 = plt.subplots(1)
plt.sca(ax41)
plt.plot(case_ma7_all.loc[K0:end].index, case_ma7_all.loc[K0:end], label='Actual Case Count', c=gold,
         linewidth=width)
plt.plot(ihme_pred.loc[start:end].index, ihme_pred.loc[start:end], label='IHME 7-day Forecast', c=gray,
         linewidth=width)
plt.plot(naive_pred.loc[start:end].index, naive_pred.loc[start:end], label='Naive Estimate', c=bright_orange,
         linewidth=width)

plt.plot(case_only_pred.loc[start:end].index, case_only_pred.loc[start:end], label='7-Day Forecast using Case Count Only',
         linewidth=width-1, c=purple)
plt.plot(smell_pred.loc[start:end].index, smell_pred.loc[start:end], label='7-Day Forecast Case Count w/ Loss of Smell/Taste',
         linewidth=width-1, c=bright_purp)
plt.plot(hh_pred.loc[start:end].index, hh_pred.loc[start:end], label='7-Day Forecast Case Count w/ Household Symptoms',
         linewidth=width-1, c=light_purp)

"""
ax42 = ax41.twinx()
plt.sca(ax42)
plt.plot(survey_data['hh_delay0'].loc[K0:end].index, survey_data['hh_delay0'].loc[K0:end], c='red',
         label='Count of Household Symptom Response', linewidth=width-1)
plt.plot(survey_data['smell_delay0'].loc[K0:end].index, survey_data['smell_delay0'].loc[K0:end], c='maroon',
         label='Count of Loss of Smell Response', linewidth=width-1)
plt.grid(axis='y', linestyle=':')

plt.ylabel('Number of Responses per Day')
plt.legend(loc='upper right')
plt.ylim(rightylim_low, rightylim_high)
"""

#plt.sca(ax41)
"""
plt.plot(predicted_case.index, predicted_case.loc[K0:], label='Our 7-Day Forecast',
         c=purple, linewidth=width)
"""



plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')

plt.legend(loc='upper right')
plt.ylim(-100, 3600)
#plt.xlim(xlim_left, xlim_right)

#plt.title('Miami-Dade, Household Symptoms, delay of 6 days')

(-100, 3600)

In [ ]:
county_preds[estimates[0]]


In [ ]:
K0

In [21]:
ihme_ny_state = pd.read_csv(r'data/seiir_projections/new_york_proj.csv', index_col='date', parse_dates=True)

In [22]:
ihme_ny_state

,S,E,I1,I2,R,beta
date,,,,,,
2020-02-10,1.979023e+07,106.854381,25.532244,0.000000,0.000000e+00,5.000033
2020-02-11,1.979019e+07,135.370375,34.165081,5.735700,1.868136e+00,3.828568
2020-02-12,1.979014e+07,163.677532,44.895678,12.645587,6.414865e+00,3.065767
2020-02-13,1.979009e+07,185.941663,55.374704,20.009029,1.385817e+01,2.621240
2020-02-14,1.979004e+07,209.578200,66.865630,28.139395,2.479945e+01,2.268072
...,...,...,...,...,...,...
2021-12-27,1.258239e+07,6212.135951,3087.113438,2385.406004,7.196296e+06,0.872742
2021-12-28,1.258052e+07,6413.925937,3187.020352,2464.107692,7.197785e+06,0.874872
2021-12-29,1.257859e+07,6622.087015,3290.015767,2545.245365,7.199323e+06,0.876996


In [29]:
gray_start = predicted_seiir_prior.index[0]
gray_end = predicted_seiir_prior.index[-1]
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.plot(ihme_ny_state.loc[gray_start:gray_end].index, ihme_ny_state['I2'].loc[gray_start:gray_end],
         label='IHME State I2')
plt.title('Nassau, b = 0.06975')
plt.legend()

In [143]:
# NYC plot settings

xlim_left = datetime.date(2020, 4, 12)
xlim_right = datetime.date(2020, 11, 3)

leftylim_low = -400
leftylim_high = 7500

rightylim_low = -3
rightylim_high = 57

xtick_size = 14
xlabel_size = 14

In [ ]:
# Nassau plot settings
xlim_left = datetime.date(2020, 4, 12)
xlim_right = datetime.date(2020, 11, 3)

leftylim_low = -66
leftylim_high = 1000

rightylim_low = -.3
rightylim_high = 8

xtick_size = 14
xlabel_size = 14

In [ ]:
# Westchester plot settings
xlim_left = datetime.date(2020, 4, 12)
xlim_right = datetime.date(2020, 11, 3)

leftylim_low = -28
leftylim_high = 700

rightylim_low = -.5
rightylim_high = 17

xtick_size = 14
xlabel_size = 14

In [ ]:
# Albany plot settings
xlim_left = datetime.date(2020, 4, 12)
xlim_right = datetime.date(2020, 11, 3)

leftylim_low = -6
leftylim_high = 110

rightylim_low = -.125
rightylim_high = 3.8

xtick_size = 14
xlabel_size = 14

In [ ]:
# Erie plot settings
xlim_left = datetime.date(2020, 4, 12)
xlim_right = datetime.date(2020, 11, 3)

leftylim_low = -10
leftylim_high = 280

rightylim_low = -.8
rightylim_high = 22

xtick_size = 14
xlabel_size = 14